In [1]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
import spacy
import sklearn

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score



In [3]:
df = pd.read_csv('NoThemeTweets.csv')
df = df.drop(["id", "tweet_date"], axis=1)
df = df[:50000] # 50k rows
df.head()

,tweet_text,sentiment,query_used
0,@Tixaa23 14 para eu ir :),Positivo,:)
1,@drexalvarez O meu like eu já dei na época :),Positivo,:)
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo,:)
3,:D que lindo dia !,Positivo,:)
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo,:)


In [13]:
lb = LabelEncoder()

df["sentiment"] = lb.fit_transform(df["sentiment"])
df["query_used"] = lb.fit_transform(df["query_used"])

,id,tweet_text,tweet_date,sentiment,query_used
0,1026606627041026050,ela vendo a família :( FUGIDINHA VERA LÚCIA,Mon Aug 06 23:11:07 +0000 2018,Negativo,vera lucia :(
1,1026266117646561281,Eu odeio o fato que muito provavelmente meu vo...,Mon Aug 06 00:38:04 +0000 2018,Negativo,ciro :(
2,1025889796097564672,"Ciro ""mita"" tanto quanto aquele outro, não que...",Sat Aug 04 23:42:42 +0000 2018,Negativo,ciro :(
3,1026960047937646598,Raiva do Bolsonaro agora :),Tue Aug 07 22:35:30 +0000 2018,Positivo,bolsonaro :)
4,1026447105932623872,Reflexos da alienação petista comandada por Lu...,Mon Aug 06 12:37:15 +0000 2018,Negativo,lula :(
...,...,...,...,...,...
61586,1055096632600080384,As concorrentes estão prontas para tudo. :p Lo...,Wed Oct 24 14:00:14 +0000 2018,Positivo,PT :)
61587,1055204789058002949,@Frases_RCampos @vrr93 Falei de um já pensando...,Wed Oct 24 21:10:01 +0000 2018,Positivo,governo :)
61588,1054982223793938433,@njg00 @christophssj @pecesiqueira @cauemoura ...,Wed Oct 24 06:25:37 +0000 2018,Positivo,governo :)
61589,1055144918451187714,que coisa mais triste uma amiga defendendo um ...,Wed Oct 24 17:12:06 +0000 2018,Negativo,fascista :(


In [15]:
df['tweet_text'] = df['tweet_text'].apply(preprocess_text)

NameError: name 're' is not defined

In [5]:
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])

In [6]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['tweet_text'])
y = df['sentiment']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
modelo = MultinomialNB()
modelo.fit(X_train, y_train)

MultinomialNB()

In [9]:
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.93      0.81       500
           1       0.91      0.63      0.74       500

    accuracy                           0.78      1000
   macro avg       0.81      0.78      0.78      1000
weighted avg       0.81      0.78      0.78      1000



O código implementa um pipeline básico para análise de sentimentos de tweets em português, utilizando TF-IDF para feature extraction e um modelo Naive Bayes para classificação. A avaliação é feita com métricas padrão de classificação. 

Os resultados obtidos pelo relatório de classificação sugerem que o modelo tem um desempenho desigual em prever sentimentos negativos (0) e positivos (1). Com uma precisão de 72% para sentimentos negativos e 91% para positivos, o modelo é mais confiável ao prever um tweet como positivo quando realmente é. No entanto, a revocação (recall) para sentimentos negativos é significativamente maior (93%) do que para positivos (63%), indicando que o modelo é mais capaz de identificar todos os casos relevantes de sentimentos negativos.

A pontuação F1, que é a média harmônica entre precisão e revocação, está mais equilibrada para sentimentos negativos (0.81) do que para positivos (0.74), refletindo uma tendência do modelo em performar melhor na identificação de tweets negativos.

Com relação a melhorias, o codigo apresentado embora faça o uso da tokenização NLTK, ele não remove stopwords, oque poderia ser relevante para uma melhora no desempenho e performance do modelo, seria uma melhoria que valeria a inclusão para fins de comparação e acompanhamento dos scores gerados. O codigo também não faz tratamento de emoji e slangs, então esses pontos poderiam ser incluidos na fase de pre processamento. Também não há um processo de feture selection, que determine quais termos são mais significativos no processo de analise de sentimentos. 

No nível de modelagem o código poderia utilizar validação cruzada para garantir que o modelo seja estável e para estimar o desempenho em uma amostra independente de dados. O código usa o modelo Naive Bayes Multinomial com os parâmetros padrão e poderia haver uma busca por hiperparâmetros para melhorar o desempenho do modelo. 

No pós processamento o relatório de classificação é impresso, mas não há uma análise subsequente dos resultados, o que poderia incluir a investigação de erros de classificação para entender onde o modelo está tendo problemas. As problemáticas com relação a reprodutibilidade e escalabilidade podem ser tomadas como o fato de o código não salvar o modelo treinado ou vetorizado, oque seria útil para aplicação do modelo a novos dados sem a necessidade de haver um retreinamento. 
